[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset
we only forcus on "labeledTrainData.csv" file

Let's first of all have a look at the data.

[Click here to download dataset](https://s3-ap-southeast-1.amazonaws.com/ml101-khanhnguyen/week3/assignment/labeledTrainData.tsv)

In [1]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [16]:
# Read dataset with extra params sep='\t', encoding="latin-1"
data = pd.read_csv("labeledTrainData.csv", sep='\t', encoding='latin-1')
data.head()

,id,sentiment,review
0,5814_8,1,With all this stuff going down at the moment w...
1,2381_9,1,"\The Classic War of the Worlds\"" by Timothy Hi..."
2,7759_3,0,The film starts with a manager (Nicholas Bell)...
3,3630_4,0,It must be assumed that those who praised this...
4,9495_8,1,Superbly trashy and wondrously unpretentious 8...


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000 entries, 0 to 24999
Data columns (total 3 columns):
id           25000 non-null object
sentiment    25000 non-null int64
review       25000 non-null object
dtypes: int64(1), object(2)
memory usage: 586.1+ KB


No missing data points

## 2. Preprocessing

In [29]:
# stop words
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Huy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [12]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

In [13]:
print(preprocessor("2343%%445@gasd@@.com"))

2343 445 gasd com 


In [15]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here
porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()] # Your code here

In [20]:
# split the dataset in train and test
# Your code here
from sklearn.model_selection import train_test_split

X = data["review"]
y = data["sentiment"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=101, stratify=y)

In [26]:
X_train.shape

(20000,)

In [27]:
y_train.shape

(20000,)

## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [32]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                       tokenizer=tokenizer_porter,
                       preprocessor=preprocessor)

clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...nalty='l2', random_state=0, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

## 4. Evaluate Model

In [35]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

predictions = clf.predict(X_test)
print("accuracy: ", accuracy_score(y_test, predictions))
print("confusing_maxtrix: \n", confusion_matrix(y_test, predictions))
print("classification_report: \n", classification_report(y_test, predictions))

accuracy:  0.8896
confusing_maxtrix: 
 [[2186  314]
 [ 238 2262]]
classification_report: 
              precision    recall  f1-score   support

          0       0.90      0.87      0.89      2500
          1       0.88      0.90      0.89      2500

avg / total       0.89      0.89      0.89      5000



## 5. Export Model 

In [37]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open("NLP-LR.pkl", 'wb'), protocol=4)

## 6. Try Decision tree algorithm

In [41]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

clf = Pipeline([('vect', tfidf),
                ('clf', DecisionTreeClassifier(random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...         min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best'))])

In [43]:
predictions = clf.predict(X_test)
print("accuracy: ", accuracy_score(y_test, predictions))
print("confusing_maxtrix: \n", confusion_matrix(y_test, predictions))
print("classification_report: \n", classification_report(y_test, predictions))

accuracy:  0.706
confusing_maxtrix: 
 [[1785  715]
 [ 755 1745]]
classification_report: 
              precision    recall  f1-score   support

          0       0.70      0.71      0.71      2500
          1       0.71      0.70      0.70      2500

avg / total       0.71      0.71      0.71      5000



Accuracy lower than using logistic regression

## Using Random-forest algorithm

In [50]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

clf = Pipeline([('vect', tfidf),
                ('clf', RandomForestClassifier(n_estimators=100, random_state=0))])
clf.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preproc...stimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False))])

In [51]:
predictions = clf.predict(X_test)
print("accuracy: ", accuracy_score(y_test, predictions))
print("confusing_maxtrix: \n", confusion_matrix(y_test, predictions))
print("classification_report: \n", classification_report(y_test, predictions))

accuracy:  0.8544
confusing_maxtrix: 
 [[2136  364]
 [ 364 2136]]
classification_report: 
              precision    recall  f1-score   support

          0       0.85      0.85      0.85      2500
          1       0.85      0.85      0.85      2500

avg / total       0.85      0.85      0.85      5000



Lower than logistic regression